In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
df = pd.read_excel(r"C:\Users\utkarsh.alpuria\Desktop\My Work\Python\Python ML\Python ML Algorithm\Deep_Learning\NN_Datasets\Insurance_NN.xlsx")
df.head()

,age,bought_insurance,affordibility
0,22,0,1
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[["age", "affordibility"]], df.bought_insurance, test_size=0.2, random_state=True)

In [6]:
# scale age from 0 to 1 for better performance of NN 
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled["age"]/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled["age"]/100

In [11]:
X_train_scaled.head()

,age,affordibility
20,0.21,1
21,0.26,0
4,0.46,1
2,0.47,1
22,0.40,1


In [24]:
# Using Tensorflow

model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation="sigmoid", kernel_initializer="ones", bias_initializer="zeros")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(X_train_scaled, y_train, epochs=5000)

Epoch 1/5000


c:\Users\utkarsh.alpuria\Desktop\My Work\Python\Python ML\Python ML Algorithm\.ml_algo_py_venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 958ms/step - accuracy: 0.5238 - loss: 0.6928
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5238 - loss: 0.6925
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5238 - loss: 0.6921
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5238 - loss: 0.6918
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5238 - loss: 0.6915
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5238 - loss: 0.6911
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5238 - loss: 0.6908
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5238 - loss: 0.6904
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5238 - loss: 0.6901
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5238 - loss: 0.6897
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5238 - loss: 0.6894
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5238 - lo

In [25]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.8333 - loss: 0.3518


[0.35176849365234375, 0.8333333134651184]

In [26]:
X_test_scaled

,age,affordibility
17,0.58,1
14,0.49,1
18,0.19,0
3,0.52,0
23,0.45,1
10,0.18,1


In [27]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


array([[0.8362848 ],
       [0.752328  ],
       [0.1399818 ],
       [0.522584  ],
       [0.70683384],
       [0.33642504]], dtype=float32)

In [28]:
# weights, bias after trining 
coef, intercept = model.get_weights()
coef, intercept 

(array([[5.7752695],
        [1.1939274]], dtype=float32),
 array([-2.9127424], dtype=float32))

In [29]:
# FROM SCRATCH PYTHON NN IMPLEMENTATION

import math
def sigmoid(x):
    return 1/(1 + math.exp(-x))

sigmoid(18)

0.9999999847700205

In [30]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept  # y = W1X1 + W2X2
    return sigmoid(weighted_sum)

In [31]:
print(prediction_function(0.47,1))
print(prediction_function(0.18,1))

0.7301850887400564
0.3364250423922265


C:\Users\utkarsh.alpuria\AppData\Local\Temp\ipykernel_16976\732956948.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1 + math.exp(-x))


**Now lets write Gradient Descent for Our NN**
- first we will write some helper function for gradient descennt

In [37]:
# Helper Loss function for Gradient Descent
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1 - y_true)*np.log(1 - y_predicted_new))


In [38]:
# Helper Sigmoid function for Gradient Descent
def sigmoid_numpy(X): # takes an np.array and return a np.array after sigmoid 
    return 1/(1 + np.exp(-X))

sigmoid_numpy(np.array([12,0,1,4]))

array([0.99999386, 0.5       , 0.73105858, 0.98201379])

In [41]:
# Gradient Descent
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold): # loss threshold is the loss we got after running 5000 epochs above(we break the function when our loss is same as loss_threshold to compare weights and bias from the keras NN to our gradient descent)
    # start with weights = 1, bias = 0, learning rate = 0.5
    w1 = w2 = 1 
    bias = 0
    rate = 0.5
    n = len(age) # n = number of samples

    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias # weighted_sum is a np_array
        y_predicted = sigmoid_numpy(weighted_sum) # y_predicted is a np.array

        loss = log_loss(y_true, y_predicted)

        # update the weights
        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted - y_true)) # derivative of loss w.r.t w1
        w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted - y_true)) # derivative of loss w.r.t w2

        bias_d = np.mean(y_predicted - y_true)

        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

        if loss <= loss_threshold:
            break
        
    return w1, w2, bias

In [43]:
gradient_descent(X_train_scaled["age"], X_train_scaled["affordibility"], y_train, 1000, 0.4139)

Epoch:0, w1:0.9842472455790087, w2:0.9464079665530514, bias:-0.10377941984476272, loss:0.692833313356495
Epoch:1, w1:0.9737019393523785, w2:0.901819292569998, bias:-0.1941226949834614, loss:0.6669868075012262
Epoch:2, w1:0.9678961307150366, w2:0.8655516223184165, bias:-0.27228270144662997, loss:0.6479712574557879
Epoch:3, w1:0.9662858146549119, w2:0.836727832246518, bias:-0.3396829886844933, loss:0.6341821996772135
Epoch:4, w1:0.9683059250372036, w2:0.8143881402662989, bias:-0.3977815252572106, loss:0.6242450184284397
Epoch:5, w1:0.9734111356690441, w2:0.7975772521197335, bias:-0.44797194385269945, loss:0.6170622732208935
Epoch:6, w1:0.981101220738761, w2:0.7854014822224361, bias:-0.4915238935420416, loss:0.6118035294692723
Epoch:7, w1:0.990933517574131, w2:0.7770596144959401, bias:-0.5295552687307755, loss:0.607865281163614
Epoch:8, w1:1.0025262624905193, w2:0.7718545570743764, bias:-0.5630265484284401, loss:0.6048220009329315
Epoch:9, w1:1.0155562852211453, w2:0.7691927081212144, bia

(5.78522702787746, 1.2016223895608482, -2.9252295884365114)

In [44]:
coef, intercept

(array([[5.7752695],
        [1.1939274]], dtype=float32),
 array([-2.9127424], dtype=float32))

**Now lets create our Neural Network**

In [53]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X["age"], X["affordibility"], y, epochs, loss_threshold)

    def predict(self, X_test):
        weighted_sum = self.w1*X_test["age"] + self.w2*X_test["affordibility"] + self.bias
        return sigmoid_numpy(weighted_sum)
    
    # Gradient Descent
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold): # loss threshold is the loss we got after running 5000 epochs above(we break the function when our loss is same as loss_threshold to compare weights and bias from the keras NN to our gradient descent)
        # start with weights = 1, bias = 0, learning rate = 0.5
        w1 = w2 = 1 
        bias = 0
        rate = 0.5
        n = len(age) # n = number of samples

        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias # weighted_sum is a np_array
            y_predicted = sigmoid_numpy(weighted_sum) # y_predicted is a np.array

            loss = log_loss(y_true, y_predicted)

            # update the weights
            w1d = (1/n)*np.dot(np.transpose(age), (y_predicted - y_true)) # derivative of loss w.r.t w1
            w2d = (1/n)*np.dot(np.transpose(affordibility), (y_predicted - y_true)) # derivative of loss w.r.t w2

            bias_d = np.mean(y_predicted - y_true)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            if i%50 == 0: # only print with interval of 50 epocs
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

            if loss <= loss_threshold:
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
                break
            
        return w1, w2, bias

In [55]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=500, loss_threshold=0.4139)

Epoch:0, w1:0.9842472455790087, w2:0.9464079665530514, bias:-0.10377941984476272, loss:0.692833313356495
Epoch:50, w1:1.7594529710279012, w2:0.9689259460978523, bias:-1.155094780434972, loss:0.5557000198664956
Epoch:100, w1:2.6375438829404354, w2:1.0957025956724966, bias:-1.604226405255885, loss:0.5158621894760659
Epoch:150, w1:3.4384770821338417, w2:1.146266255532715, bias:-1.9616006430279693, loss:0.48484283332596917
Epoch:200, w1:4.163563743799218, w2:1.1692406944230131, bias:-2.2656954741145086, loss:0.4599882794738117
Epoch:250, w1:4.819184038152194, w2:1.1829408507234451, bias:-2.533753462356547, loss:0.4398335666908479
Epoch:300, w1:5.412970753272281, w2:1.1941477604391313, bias:-2.7744832785151825, loss:0.4233454293323272
Epoch:334, w1:5.78522702787746, w2:1.2016223895608482, bias:-2.9252295884365114, loss:0.41382357064251674


In [56]:
customModel.predict(X_test_scaled)

17    0.836419
14    0.752344
18    0.138711
3     0.520760
23    0.706769
10    0.335756
dtype: float64

In [57]:
model.predict(X_test_scaled) # keras model prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[0.8362848 ],
       [0.752328  ],
       [0.1399818 ],
       [0.522584  ],
       [0.70683384],
       [0.33642504]], dtype=float32)